# 加法示例 - CUDA
参考：
- https://tvm.apache.org/docs/dev/tutorial/codebase_walkthrough.html
- gallery/tutorial/tensor_expr_get_started.py
- coureDLC,7
- tvm\tests\python\unittest\test_lower_build.py

import os
import sys

```
os.environ['PYTHONPATH']='D:\\Dev\\tvm\\python'
print(os.environ.get('PYTHONPATH'))

print(os.environ.get('PATH'))

print(sys.path)

print(os.environ.get('PYTHONPATH'))

print(sys.path)



sys.path.pop()

print(sys.path)

os.environ['PATH'] += ';D:\\Dev\\tvm\\build\\Release'
```


In [46]:
import tvm
from tvm import te
from tvm.ir.module import IRModule

In [47]:
import numpy as np

In [48]:
n = 1024

target = "cuda"

In [49]:
dev = tvm.device(target, 0)

# declare the computation using the expression API
A = te.placeholder((n, ), name="A")
B = te.placeholder((n, ), name="B")
C = te.compute((n,), lambda i: A[i] + B[i], name="C")

# Default schedule
func = te.create_prim_func([A, B, C])
func = func.with_attr("global_symbol", "main")
ir_module = IRModule({"main": func})
print(ir_module.script())

# from tvm.script import tir as T
@tvm.script.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer[1024, "float32"], B: T.Buffer[1024, "float32"], C: T.Buffer[1024, "float32"]):
        # function attr dict
        T.func_attr({"global_symbol": "main", "tir.noalias": True})
        # body
        # with T.block("root")
        for i in T.serial(1024):
            with T.block("C"):
                v_i = T.axis.spatial(1024, i)
                T.reads(A[v_i], B[v_i])
                T.writes(C[v_i])
                C[v_i] = A[v_i] + B[v_i]
    



In [50]:
tvm.tir.Schedule??

In [51]:
# Construct a TensorIR schedule class from an IRModule
sch = tvm.tir.Schedule(ir_module) 
# Get block by its name
block_c = sch.get_block("C")
# Get loops surronding the block
(i,) = sch.get_loops(block_c)

In [52]:
sch.bind(i, "threadIdx.x")
print(sch.mod.script())

# from tvm.script import tir as T
@tvm.script.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer[1024, "float32"], B: T.Buffer[1024, "float32"], C: T.Buffer[1024, "float32"]):
        # function attr dict
        T.func_attr({"global_symbol": "main", "tir.noalias": True})
        # body
        # with T.block("root")
        for i in T.thread_binding(1024, thread="threadIdx.x"):
            with T.block("C"):
                v_i = T.axis.spatial(1024, i)
                T.reads(A[v_i], B[v_i])
                T.writes(C[v_i])
                C[v_i] = A[v_i] + B[v_i]
    



这里的线程绑定操作，最终到cuda代码的映射操作是在代码生成阶段完成。
```c++
// src/target/source/codegen_cuda.cc

class ThreadIdxExtractor : public tir::StmtVisitor {
 private:
  void VisitStmt_(const AttrStmtNode* op) final {
    if (op->attr_key == tir::attr::thread_extent) {
      IterVar iv = Downcast<IterVar>(op->node);
      if (iv->var->name_hint == "threadIdx.x" || iv->thread_tag == "threadIdx.x") {
        threadIdx_x_ext = op->value;
      }
      if (iv->var->name_hint == "threadIdx.y" || iv->thread_tag == "threadIdx.y") {
        threadIdx_y_ext = op->value;
      }
      if (iv->var->name_hint == "threadIdx.z" || iv->thread_tag == "threadIdx.z") {
        threadIdx_z_ext = op->value;
      }
    }
    StmtVisitor::VisitStmt_(op);
  }

 public:
  PrimExpr threadIdx_x_ext = Integer(1);
  PrimExpr threadIdx_y_ext = Integer(1);
  PrimExpr threadIdx_z_ext = Integer(1);
};
```


In [53]:
fadd = tvm.build(sch.mod, target=target)

In [54]:
fadd.imported_modules

[Module(cuda, 18370048f78)]

In [55]:
print(fadd.imported_modules[0].get_source())


#ifdef _WIN32
  using uint = unsigned int;
  using uchar = unsigned char;
  using ushort = unsigned short;
  using int64_t = long long;
  using uint64_t = unsigned long long;
#else
  #define uint unsigned int
  #define uchar unsigned char
  #define ushort unsigned short
  #define int64_t long long
  #define uint64_t unsigned long long
#endif
extern "C" __global__ void __launch_bounds__(1024) main_kernel0(float* __restrict__ C, float* __restrict__ A, float* __restrict__ B) {
  C[((int)threadIdx.x)] = (A[((int)threadIdx.x)] + B[((int)threadIdx.x)]);
}




In [56]:
type(fadd)

tvm.driver.build_module.OperatorModule

In [57]:
a = tvm.nd.array(np.random.uniform(size=n).astype(A.dtype), dev) #在目标设备分配相应空间，并将数据填入。
b = tvm.nd.array(np.random.uniform(size=n).astype(B.dtype), dev)
c = tvm.nd.array(np.zeros(n, dtype=C.dtype), dev)
fadd(a, b, c)
output = c.numpy()

In [58]:
evaluator = fadd.time_evaluator(fadd.entry_name, dev, number=1)
print("Baseline: %f" % evaluator(a, b, c).mean)

Baseline: 0.000034


In [59]:
tgt_nvptx = tvm.target.Target("nvptx")

In [60]:
fdd_nvptx = tvm.build(sch.mod, target=tgt_nvptx)

In [61]:
fdd_nvptx.imported_modules[0].get_source()

'; ModuleID = \'TVMPTXModule\'\nsource_filename = "TVMPTXModule"\ntarget datalayout = "e-i64:64-i128:128-v16:16-v32:32-n16:32:64"\ntarget triple = "nvptx64-nvidia-cuda"\n\n; Function Attrs: mustprogress nofree nosync nounwind willreturn\ndefine dllexport void @main_kernel0(float* noalias nocapture writeonly %C, float* noalias nocapture readonly %A, float* noalias nocapture readonly %B) local_unnamed_addr #0 {\nentry:\n  %0 = tail call i32 @llvm.nvvm.read.ptx.sreg.tid.x(), !range !2\n  %1 = zext i32 %0 to i64\n  %2 = getelementptr inbounds float, float* %B, i64 %1\n  %3 = load float, float* %2, align 4, !tbaa !3\n  %4 = getelementptr inbounds float, float* %A, i64 %1\n  %5 = load float, float* %4, align 4, !tbaa !6\n  %6 = fadd float %3, %5\n  %7 = getelementptr inbounds float, float* %C, i64 %1\n  store float %6, float* %7, align 4, !tbaa !8\n  ret void\n}\n\n; Function Attrs: nofree nosync nounwind readnone speculatable\ndeclare i32 @llvm.nvvm.read.ptx.sreg.tid.x() #1\n\n; Function At

In [62]:
evaluator = fadd.time_evaluator(fdd_nvptx.entry_name, dev, number=1)
print("Baseline: %f" % evaluator(a, b, c).mean)

Baseline: 0.000035


# 矩阵乘法 - CUDA

# 加法示例- SDAA

In [63]:
target_cuda = tvm.target.cuda()

In [64]:
target_cuda

cuda -keys=cuda,gpu -arch=sm_75 -max_num_threads=1024 -model=unknown -thread_warp_size=32

In [65]:
tgt, t_host = target_cuda.canon_target_and_host(target_cuda)

In [66]:
print(tgt)

cuda -keys=cuda,gpu -arch=sm_75 -max_num_threads=1024 -model=unknown -thread_warp_size=32
